Software installed:
Pandas, numpy, matplotlib, and scipy were all installed via the Terminal in my root directory using the 'conda install package_name -y' command. Plotly and cufflinks were installed via the Terminal and in my root directory using the 'pip install package_name' command.
The plotly package is of interest because it can be used both online and offline to generate a broad range interactive plots with native java script capabilities in Jupyter notebook. 2-D and 3-D biological datasets can be plotted using this package, making it useful for plotting trends in clinical data or creating layered heatmaps of multidimensional data such as air quality.

In [40]:
#imports
#terminal conda install of pandas and matplotlib. pip install plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import ipywidgets as widgets
#from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from IPython.display import display, clear_output, Image
import plotly as py
import plotly.offline as plotly_offline
import plotly.plotly as plotly
import plotly.tools as tls
import plotly.graph_objs as go
#import cufflinks as cf #bind pandas df with plotly
from scipy import special

py.offline.init_notebook_mode(connected=True)


Read in and verify correct import of desired dataset. Text file was created in the same folder as the calling notebook. In u6015361 DBMI_6018_2018/FinalExam folder online.

In [41]:
#read in vcf as csv in current folder

vcf = pd.read_csv('example_data.txt', delimiter = "\t", usecols=range(0,10)) #read in tab delimited file
vcf.head(5) #make sure file loaded correctly

,Chr,Position,Ref,Alt,Consequence,Transcript Consequence,Annotation,Allele Count,Allele Number,Allele Frequency
0,16,68771248,C,G,c.-71C>G,c.-71C>G,5_prime_UTR,343,31362,0.010937
1,16,68771254,C,G,c.-65C>G,c.-65C>G,5_prime_UTR,1,31370,0.000032
2,16,68771256,C,A,c.-63C>A,c.-63C>A,5_prime_UTR,1,31386,0.000032
3,16,68771265,G,C,c.-54G>C,c.-54G>C,5_prime_UTR,57,31376,0.001817
4,16,68771266,G,GCCCGA,c.-45_-41dupCGACC,c.-45_-41dupCGACC,5_prime_UTR,7,31376,0.000223


In [42]:
vcf.shape

(300, 10)

In [59]:
def find_start(vcf, col, index=0):
    """locate begining of intron based on presence of + or - in consequence"""
    while True:
        if not ('+' in vcf[col][index] or '-' in vcf[col][index]):
            index += 1
        else:
            return index
        

In [60]:
#look at base positions to define intron and exon boundaries
def find_end(vcf, col, index=0):
    """locate begining of intron based on presence of + or - in consequence"""
    while True:
        if '+' in vcf[col][index] or '-' in vcf[col][index]:
            index +=1
        else:
            return index-1

In [45]:
find_end(vcf, 'Transcript Consequence')

21

In [46]:
#vcf.Position[0:22]

In [47]:
def get_width(vcf, pos_col='Position', search_col='Transcript Consequence', start_loc=0):
    """calculate the span of the intron based on _end locations"""
    start_index = find_start(vcf, search_col, index=start_loc)
    end_index = find_end(vcf, search_col, index=start_index)
    return vcf[pos_col][end_index] - vcf[pos_col][start_index], start_index, end_index
    return start_index, end_index



In [48]:
rslt = get_width(vcf)
rslt

(63, 0, 21)

In [49]:
get_width(vcf, start_loc = rslt[2]+1)

(826, 37, 69)

In [52]:
"""Define the boundaries of the introns. Also infers the boundaries of the exons"""
start_loc = 0
try:
    while start_loc < vcf.shape[0]:
        rslt = get_width(vcf, start_loc=start_loc)
        print(rslt)
        start_loc = rslt[2]+1
except Exception as error:
    print(error)


(63, 0, 21)
(826, 37, 69)
(63253, 93, 122)
(6522, 195, 233)
(110, 260, 287)
300


In [ ]:
vcf.Position.iloc[-1]-vcf.Position.iloc[288]

in reference column, calculate number of times each base occurs. Plot frequency of each to determine if any bases
mutate at a higher frequency.  
use plotly interactive offline bar chart to view frequency rate to visualize mutation biases.

In [25]:
#count kmer occurrences
base_counts = Counter()
vcf['Ref'].str.upper().str.split().apply(base_counts.update)
#print(base_counts)

#Return a list of the n most common elements and their counts from the most common to the least. 
base_counts.most_common()
#convert to datafram from dict
d = base_counts
base_counts_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
base_counts_df = base_counts_df.rename(columns={'index':'kmer', 0:'count'})

#calculate and round frequencies
base_counts_df['frequency']= (base_counts_df['count']/base_counts_df['count'].sum())*100
base_counts_df.frequency = base_counts_df.frequency.round(1)
#verify new df structure and data accuracy
base_counts_df


,kmer,count,frequency
0,C,121,40.3
1,G,88,29.3
2,A,38,12.7
3,T,48,16.0
4,CCT,1,0.3
5,AG,1,0.3
6,ATCT,1,0.3
7,TTC,1,0.3
8,CTCTGTT,1,0.3


In [26]:
#create offline plotly bar graph to compare frequencies of reference mutation types.
"""hover over bar graph for frequency value of each reference mutation"""
N = 9
x = np.linspace(0, 9, N)
y = np.random.randn(N)
base_counts_dfgraph = pd.DataFrame({'kmer': x, 'frequency': y})

graph = [
    go.Bar(
        x=base_counts_df['kmer'], # assign x as the dataframe column 'kmer'
        y=base_counts_df['frequency'] # assign y as the dataframe column 'frequency'
    )
]

# IPython notebook
py.offline.iplot(graph, filename='kmers-pandas-bar-chart')

view and sort the dataframe by reference base, selecting one or multiple kmer types at a time. can use this approach to build an interactive plot to visualize different properties of the dataframe- kmer mutation frequencies and other defining features of the df columns.

In [55]:
vcf[vcf.Ref.isin(('A','C'))]

,Chr,Position,Ref,Alt,Consequence,Transcript Consequence,Annotation,Allele Count,Allele Number,Allele Frequency
0,16,68771248,C,G,c.-71C>G,c.-71C>G,5_prime_UTR,343,31362,0.010937
1,16,68771254,C,G,c.-65C>G,c.-65C>G,5_prime_UTR,1,31370,0.000032
2,16,68771256,C,A,c.-63C>A,c.-63C>A,5_prime_UTR,1,31386,0.000032
5,16,68771268,C,T,c.-51C>T,c.-51C>T,5_prime_UTR,1,120226,0.000008
6,16,68771268,C,G,c.-51C>G,c.-51C>G,5_prime_UTR,1,120226,0.000008
9,16,68771278,C,A,c.-41C>A,c.-41C>A,5_prime_UTR,4,123766,0.000032
10,16,68771281,A,AC,c.-35dupC,c.-35dupC,5_prime_UTR,3,124156,0.000024
11,16,68771284,C,T,c.-35C>T,c.-35C>T,5_prime_UTR,1,124726,0.000008
13,16,68771287,C,T,c.-32C>T,c.-32C>T,5_prime_UTR,3,125000,0.000024
14,16,68771287,C,G,c.-32C>G,c.-32C>G,5_prime_UTR,8,125000,0.000064


Beginning build of an interactive plot with chromosome location as x-axis values and mutation distribution as y-axis values. The current version contains an interactive dropdown list of the unique kmers in the df, but currently only displays the dropdown selection(s) in df form.

In [57]:
x = np.linspace(0, np.pi, 1000)

layout = go.Layout(
    title = 'Kmer Distribution',
    yaxis = dict(
        title = 'Kmer Count'
    ),
    xaxis = dict(
        title = 'Chr 16 Position'
    )
)

def update_plot1(signals):
    vcf[vcf.Ref in signals]
    for s in signals:
        trace1 = go.Scatter(
            x = x,
            y = special.jv(s, freq * x),
            mode = 'lines',
            name = 'bessel {}'.format(s),
            line = dict(
                shape = 'spline'
            )
        )
        data.append(trace1)
    fig = go.Figure(data = data, layout = layout)
    py.offline.iplot(fig)

def update_plot(signals):
    selected = vcf[vcf.Ref.isin(signals)]
    display(selected)
    
signals = widgets.SelectMultiple(options = list(vcf['Ref'].unique()), value = ("C", ), description = 'Kmer Selection')
#freq = widgets.FloatSlider(min=1, max=20, value=1, description='Freq')
widgets.interactive(update_plot, signals=signals)



interactive(children=(SelectMultiple(description='Kmer Selection', index=(0,), options=('C', 'G', 'A', 'T', 'C…

References

Matplotlib for frequency plots: https://matplotlib.org/users/pyplot_tutorial.html
while loop: https://wiki.python.org/moin/WhileLoop
Counter() from collections for bases: https://pymotw.com/2/collections/counter.html
ipywidgets interact: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
plotly tutorial: https://plot.ly/python/getting-started/
plotly stacked bar chart for pandas bar chart: https://plot.ly/pandas/bar-charts/
counter for bases: https://stackoverflow.com/questions/18936957/count-distinct-words-from-a-pandas-data-frame
order base counts of Counter dict: https://stackoverflow.com/questions/20316299/formatting-output-of-counter
counter object into pd dataframe: https://stackoverflow.com/questions/31111032/transform-a-counter-object-into-a-pandas-dataframe
calculate base percentage: https://stackoverflow.com/questions/23539832/how-to-calculate-percentage-with-pandas-dataframe
round base frequencies: https://stackoverflow.com/questions/26133538/round-a-single-column-in-pandas
dropdown widget: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html